<a href="https://colab.research.google.com/github/czarny1p/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [20]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/Matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/Matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]

  )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),                 
  ])

#get_cnn_v4(input_shape, num_classes).summary()
#train_and_predict( get_cnn_v5(input_shape, num_classes) )

In [16]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 22ms/step - loss: 2.4783 - accuracy: 0.3131 - val_loss: 0.9136 - val_accuracy: 0.7303
Epoch 2/5
272/272 [==============================] - 6s 21ms/step - loss: 0.8063 - accuracy: 0.7454 - val_loss: 0.2142 - val_accuracy: 0.9520
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3729 - accuracy: 0.8852 - val_loss: 0.0751 - val_accuracy: 0.9776
Epoch 4/5
272/272 [==============================] - 6s 21ms/step - loss: 0.2455 - accuracy: 0.9245 - val_loss: 0.0476 - val_accuracy: 0.9880
Epoch 5/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1885 - accuracy: 0.9449 - val_loss: 0.0286 - val_accuracy: 0.9922


0.9707482993197278

In [17]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.1565 - accuracy: 0.9707


[0.1564851999282837, 0.9707483053207397]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),                 
  ])

#get_cnn_v4(input_shape, num_classes).summary()
#train_and_predict( get_cnn_v5(input_shape, num_classes) )

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train,
      y_train,
      batch_size=128,#int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [31]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.3158762714085984, 'dropout_cnn_block_three': 0.410960444323035, 'dropout_cnn_block_two': 0.3199971320292658, 'dropout_dense_block_one': 0.6897726114281911, 'dropout_dense_block_two': 0.477158356462683}
accuracy=0.9263038635253906
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.41705616359948866, 'dropout_cnn_block_three': 0.38879688872122864, 'dropout_cnn_block_two': 0.43746310924660026, 'dropout_dense_block_one': 0.46281998943793656, 'dropout_dense_block_two': 0.512490735642855}
accuracy=0.9301587343215942
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.42881115815235094, 'dropout_cnn_block_three': 0.3510973694359794, 'dropout_cnn_block_two': 0.4412327004196013, 'dropout_dense_block_one': 0.4377388762905726, 'dropout_dense_block_two': 0.4255608687804069}
accuracy=0.9396825432777405
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.41453656610847894, 'dropout_cnn_block_three': 0.3397144126920032, 'dropout_cnn_block_two': 0.4049651607722